<a href="https://colab.research.google.com/github/vfrantc/dehaze_detect/blob/main/evaluate_detection_playground3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Nov 24 08:45:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# RUN DETECTOR ON UNPROCESSED IMAGES

In [3]:
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

     |████████████████████████████████| 390 kB 8.4 MB/s 
     |████████████████████████████████| 185 kB 62.8 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.17-cp37-cp37m-linux_x86_64.whl size=44591275 sha256=46bbf34fe8d424c04126649c59c735c8aca6ef4514f169cef6a6ade813d1e7bb
  Stored in directory: /root/.cache/pip/wheels/5f/63/2c/49cc449e4a860b364c49c0b77d2275cb012f625d7c9203e444
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 21762, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 21762 (delta 16), reused 31 (delta 8), pack-reused 21699
Receiving objects: 100% (21762/21762), 25.23 MiB | 31.32 MiB/s, done.
Resolving deltas: 100% (15259/15259), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=3041617a3397901e41f82b212af19aa0200c2a938f58301de6eda463f075c944
 

In [4]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth \
      -O checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth

--2021-11-24 09:20:05--  https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.59
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.59|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611860651 (584M) [application/octet-stream]
Saving to: ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’

checkpoints/scnet_x 100%[===================>] 583.52M  6.64MB/s    in 82s     

2021-11-24 09:21:28 (7.10 MB/s) - ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’ saved [611860651/611860651]



In [ ]:
#%cd mmdetection

[Errno 2] No such file or directory: 'mmdetection'
/content/mmdetection


In [5]:
from glob import glob
import os

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

config = 'configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py'
checkpoint = 'checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth'
model = init_detector(config, checkpoint, device='cuda:0')
model = MMDataParallel(model, device_ids=[0])

load checkpoint from local path: checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth


In [6]:
def create_dataset(folder):
  cfg = Config.fromfile('./configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py')
  cfg.dataset_type = 'CocoDataset'
  PREFIX = folder
  cfg.data.test.img_prefix = folder
  cfg.data.test.ann_file = os.path.join(PREFIX, 'new_test.json')
  cfg.data.test.type = 'CocoDataset'
  dataset = build_dataset(cfg.data.test)
  #dataset.CLASSES = classes
  #dataset.cat_ids = list(range(len(dataset.CLASSES)))
  data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)
  return dataset, data_loader

In [7]:
!cp /content/drive/MyDrive/msnet.zip .
!unzip -q msnet.zip 
!cp /content/drive/MyDrive/hazy_detect/out_refined.tar.gz .
!tar xzvf out_refined.tar.gz

out_refined/
out_refined/JPEGImages/
out_refined/JPEGImages/SFC_Google_182_dehz.png
out_refined/JPEGImages/GSGL_Baidu_1177_dehz.png
out_refined/JPEGImages/HEB_Baidu_199_dehz.png
out_refined/JPEGImages/JL_Google_011_dehz.png
out_refined/JPEGImages/hv42_394_dehz.png
out_refined/JPEGImages/hv39_273_dehz.png
out_refined/JPEGImages/QD_Bing_179_dehz.png
out_refined/JPEGImages/AM_Google_494_dehz.png
out_refined/JPEGImages/hv39_115_dehz.png
out_refined/JPEGImages/HEB_Bing_237_dehz.png
out_refined/JPEGImages/HF_Google_511_dehz.png
out_refined/JPEGImages/BJ_Baidu_095_dehz.png
out_refined/JPEGImages/MLS_Google_201_dehz.png
out_refined/JPEGImages/CC_Baidu_228_dehz.png
out_refined/JPEGImages/BJ_Baidu_220_dehz.png
out_refined/JPEGImages/GSGL_Google_371_dehz.png
out_refined/JPEGImages/GSGL_Baidu_1034_dehz.png
out_refined/JPEGImages/JL_Bing_751_dehz.png
out_refined/JPEGImages/YSM_Google_405_dehz.png
out_refined/JPEGImages/hv29_82_dehz.png
out_refined/JPEGImages/GSGL_Google_121_dehz.png
out_refined/JPE

In [8]:
!mkdir msnet
!mv outdoor msnet/JPEGImages
!cp /content/drive/MyDrive/new_test.json msnet

In [ ]:
# !cp /content/drive/MyDrive/hazy_detect/wavelet_unet.zip .
# !unzip -q wavelet_unet.zip 

In [ ]:
# !cp /content/drive/MyDrive/hazy_detect/out_gca.zip .
# !cp /content/drive/MyDrive/hazy_detect/dcp_coco.zip . 
# !cp /content/drive/MyDrive/hazy_detect/out_refined.tar.gz .
# !cp /content/drive/MyDrive/hazy_detect/dehazenet_coco.zip .
# !cp /content/drive/MyDrive/hazy_detect/ide_coco.zip .
# !cp /content/drive/MyDrive/hazy_detect/out_epdn.tar.gz .

In [ ]:
# !cp /content/drive/MyDrive/new_detect/aod_coco.zip . 
# !unzip -q aod_coco.zip

In [ ]:
# !unzip -q out_gca.zip
# !unzip -q dcp_coco.zip
# !tar xzvf out_refined.tar.gz
# !unzip -q dehazenet_coco.zip
# !unzip -q ide_coco.zip
# !tar xzvf out_epdn.tar.gz

out_refined/
out_refined/new_test.json
out_refined/JPEGImages/
out_refined/JPEGImages/BD_Baidu_338_dehz.png
out_refined/JPEGImages/AM_Bing_640_dehz.png
out_refined/JPEGImages/BD_Google_396_dehz.png
out_refined/JPEGImages/BD_Baidu_129_dehz.png
out_refined/JPEGImages/BD_Google_598_dehz.png
out_refined/JPEGImages/BD_Baidu_081_dehz.png
out_refined/JPEGImages/BJ_Baidu_1039_dehz.png
out_refined/JPEGImages/BJ_Baidu_1158_dehz.png
out_refined/JPEGImages/BD_Baidu_133_dehz.png
out_refined/JPEGImages/BD_Bing_303_dehz.png
out_refined/JPEGImages/BD_Bing_664_dehz.png
out_refined/JPEGImages/BD_Baidu_000_dehz.png
out_refined/JPEGImages/BD_Baidu_116_dehz.png
out_refined/JPEGImages/BD_Google_378_dehz.png
out_refined/JPEGImages/BD_Baidu_022_dehz.png
out_refined/JPEGImages/BD_Baidu_102_dehz.png
out_refined/JPEGImages/BD_Baidu_428_dehz.png
out_refined/JPEGImages/BJ_Baidu_095_dehz.png
out_refined/JPEGImages/BD_Google_249_dehz.png
out_refined/JPEGImages/AM_Google_244_dehz.png
out_refined/JPEGImages/BD_Baidu_4

In [9]:
import numpy as np
def fix_outputs(fix_outputs):
    new_outputs = []
    for output in outputs:
        new_output = []
        for i in range(80):
            new_output.append(np.array([], dtype=np.float32).reshape(0, 5))
        for i in [0, 1, 2, 3, 5]:
            new_output[i] = output[0][i]
        new_output[2] = np.vstack([new_output[2], output[0][i]])
        new_outputs.append(new_output)
    return new_outputs

# MSNET

In [10]:
method = 'msnet'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.72s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2162s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.35s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=55.18s).
Accumulating evaluation results...
DONE (t=6.96s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.356
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.566
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.384
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.339
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.473
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.

# NL

In [ ]:
method = 'nl'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

mkdir: cannot create directory ‘out_nl’: File exists
loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2317s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=73.12s).
Accumulating evaluation results...
DONE (t=8.93s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.555
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.381
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.133
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.478
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=

In [ ]:
method = 'wavelet_unet'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.37s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2144s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.85s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=54.56s).
Accumulating evaluation results...
DONE (t=6.75s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.350
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.558
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.371
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.146
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.332
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.467
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.434
 Average Recall     (AR) @[ IoU=0.50:0.

### gca

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_gca

In [ ]:
method = 'out_gca'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
outputs = single_gpu_test(model, data_loader, False, '', 0.3)

mkdir: cannot create directory ‘out_out_gca’: File exists
loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.8 task/s, elapsed: 2363s, ETA:     0s

In [ ]:
# how to post-process, and how to change the json

In [ ]:
# post-process the detection to make sure that we use only the correct ones

In [ ]:
dataset.cat_ids.append(91)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')


Evaluating bbox...
Loading and preparing results...
DONE (t=1.20s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=53.88s).
Accumulating evaluation results...
DONE (t=6.96s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.363
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.567
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.390
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.476
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.449
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.449
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.449
 Average Recall     (AR) @[ IoU=0.50:0.95

In [ ]:
!tar czf detected_gca.tar.gz out_out_gca/
!cp detected_gca.tar.gz /content/drive/MyDrive/new_detect

### dcp


In [ ]:
method = 'dcp_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.87s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2278s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.19s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=53.21s).
Accumulating evaluation results...
DONE (t=6.77s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.560
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.382
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.144
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.335
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.475
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_dcp.tar.gz out_dcp_coco/
# !cp -r detected_dcp.tar.gz /content/drive/MyDrive/new_detect

### Refined

In [11]:
!cp /content/drive/MyDrive/new_test.json out_refined

In [12]:
!ls out_refined/JPEGImages

AM_Bing_211_dehz.png	    HazyDr_Google_188_dehz.png	MLS_Bing_034_dehz.png
AM_Bing_217_dehz.png	    HazyDr_Google_240_dehz.png	MLS_Bing_036_dehz.png
AM_Bing_222_dehz.png	    HazyDr_Google_277_dehz.png	MLS_Bing_046_dehz.png
AM_Bing_229_dehz.png	    HazyDr_Google_289_dehz.png	MLS_Bing_068_dehz.png
AM_Bing_232_dehz.png	    HazyDr_Google_298_dehz.png	MLS_Bing_085_dehz.png
AM_Bing_242_dehz.png	    HazyDr_Google_324_dehz.png	MLS_Bing_095_dehz.png
AM_Bing_243_dehz.png	    HazyDr_Google_359_dehz.png	MLS_Bing_108_dehz.png
AM_Bing_274_dehz.png	    HazyDr_Google_384_dehz.png	MLS_Bing_115_dehz.png
AM_Bing_318_dehz.png	    HazyDr_Google_475_dehz.png	MLS_Bing_118_dehz.png
AM_Bing_422_dehz.png	    HazyDr_Google_476_dehz.png	MLS_Bing_123_dehz.png
AM_Bing_464_dehz.png	    HazyDr_Google_602_dehz.png	MLS_Bing_130_dehz.png
AM_Bing_536_dehz.png	    HazyDr_Google_622_dehz.png	MLS_Bing_132_dehz.png
AM_Bing_640_dehz.png	    HazyDr_Google_647_dehz.png	MLS_Bing_141_dehz.png
AM_Bing_727_dehz.png	    HazyDr_Google

In [17]:
!ls out_refined/JPEGImages 

AM_Bing_211_dehz.png	    HazyDr_Google_188_dehz.png	MLS_Bing_034_dehz.png
AM_Bing_217_dehz.png	    HazyDr_Google_240_dehz.png	MLS_Bing_036_dehz.png
AM_Bing_222_dehz.png	    HazyDr_Google_277_dehz.png	MLS_Bing_046_dehz.png
AM_Bing_229_dehz.png	    HazyDr_Google_289_dehz.png	MLS_Bing_068_dehz.png
AM_Bing_232_dehz.png	    HazyDr_Google_298_dehz.png	MLS_Bing_085_dehz.png
AM_Bing_242_dehz.png	    HazyDr_Google_324_dehz.png	MLS_Bing_095_dehz.png
AM_Bing_243_dehz.png	    HazyDr_Google_359_dehz.png	MLS_Bing_108_dehz.png
AM_Bing_274_dehz.png	    HazyDr_Google_384_dehz.png	MLS_Bing_115_dehz.png
AM_Bing_318_dehz.png	    HazyDr_Google_475_dehz.png	MLS_Bing_118_dehz.png
AM_Bing_422_dehz.png	    HazyDr_Google_476_dehz.png	MLS_Bing_123_dehz.png
AM_Bing_464_dehz.png	    HazyDr_Google_602_dehz.png	MLS_Bing_130_dehz.png
AM_Bing_536_dehz.png	    HazyDr_Google_622_dehz.png	MLS_Bing_132_dehz.png
AM_Bing_640_dehz.png	    HazyDr_Google_647_dehz.png	MLS_Bing_141_dehz.png
AM_Bing_727_dehz.png	    HazyDr_Google

In [19]:
!ls out_refined/JPEGImages | wc -l

4322


In [20]:
!ls msnet/JPEGImages | wc -l

4322


In [23]:
!ls out_refined/JPEGImages

AM_Bing_211_dehz.png	    HazyDr_Google_188_dehz.png	MLS_Bing_034_dehz.png
AM_Bing_217_dehz.png	    HazyDr_Google_240_dehz.png	MLS_Bing_036_dehz.png
AM_Bing_222_dehz.png	    HazyDr_Google_277_dehz.png	MLS_Bing_046_dehz.png
AM_Bing_229_dehz.png	    HazyDr_Google_289_dehz.png	MLS_Bing_068_dehz.png
AM_Bing_232_dehz.png	    HazyDr_Google_298_dehz.png	MLS_Bing_085_dehz.png
AM_Bing_242_dehz.png	    HazyDr_Google_324_dehz.png	MLS_Bing_095_dehz.png
AM_Bing_243_dehz.png	    HazyDr_Google_359_dehz.png	MLS_Bing_108_dehz.png
AM_Bing_274_dehz.png	    HazyDr_Google_384_dehz.png	MLS_Bing_115_dehz.png
AM_Bing_318_dehz.png	    HazyDr_Google_475_dehz.png	MLS_Bing_118_dehz.png
AM_Bing_422_dehz.png	    HazyDr_Google_476_dehz.png	MLS_Bing_123_dehz.png
AM_Bing_464_dehz.png	    HazyDr_Google_602_dehz.png	MLS_Bing_130_dehz.png
AM_Bing_536_dehz.png	    HazyDr_Google_622_dehz.png	MLS_Bing_132_dehz.png
AM_Bing_640_dehz.png	    HazyDr_Google_647_dehz.png	MLS_Bing_141_dehz.png
AM_Bing_727_dehz.png	    HazyDr_Google

In [21]:
from glob import glob
from shutil import move

In [24]:
move?

In [26]:
for imname in glob('out_refined/JPEGImages/*.png'):
  new_imname = imname.replace('_dehz.png', '.png')
  move(imname, new_imname)

In [27]:
method = 'out_refined'
!rm -rf out_out_refined
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.65s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2147s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.28s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=63.32s).
Accumulating evaluation results...
DONE (t=8.82s).

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.550
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.362
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.479
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.428
 Average Recall     (AR) @[ IoU=0.50:0.

In [ ]:
# !tar czf detected_refined.tar.gz out_out_refined/
# !cp -r detected_refined.tar.gz /content/drive/MyDrive/new_detect

# Dehazenet

In [ ]:
method = 'dehazenet_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2231s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.28s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=54.64s).
Accumulating evaluation results...
DONE (t=6.61s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.351
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.561
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.375
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.143
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.333
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.470
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.436
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_dehazenet.tar out_dehazenet/
# !cp -r detected_dehazenet.zip /content/drive/MyDrive/new_detect

# IDE

In [ ]:
method = 'dide_coco'
!cp /content/drive/MyDrive/new_test.json dide_coco
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2221s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.25s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=54.63s).
Accumulating evaluation results...
DONE (t=7.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.353
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.559
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.382
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.140
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.336
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.473
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_ide.tar.gz out_ide_coco/
# !cp -r detected_ide.tar.gz /content/drive/MyDrive/new_detect

# RTTS

In [ ]:
!rm -rf rtts_coco
!cp /content/drive/MyDrive/haze/data/RTTS.zip .
!unzip -q RTTS.zip
!mv RTTS rtts_coco 
!cp /content/drive/MyDrive/new_test.json rtts_coco

In [ ]:
method = 'rtts_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.16s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2174s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.16s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=55.34s).
Accumulating evaluation results...
DONE (t=6.71s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.354
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.562
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.378
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.337
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.473
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.441
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
# !tar czf detected_rtts_coco.tar.gz out_rtts_coco/
# !cp -r detected_rtts_coco.tar.gz /content/drive/MyDrive/new_detect

# AOD

In [ ]:
method = 'aod_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.9 task/s, elapsed: 2254s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=1.22s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=56.04s).
Accumulating evaluation results...
DONE (t=6.96s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.334
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.540
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.131
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.317
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.453
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.9

# EPDN

In [ ]:
method = 'out_epdn'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, False, '', 0.3)
metric = dataset.evaluate(fix_outputs(outputs), metric='bbox')

loading annotations into memory...
Done (t=1.90s)
creating index...
index created!
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 2.0 task/s, elapsed: 2203s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.91s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=55.97s).
Accumulating evaluation results...
DONE (t=7.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.202
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.442
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.160
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.012
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.389
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.296
 Average Recall     (AR) @[ IoU=0.50:0.9

In [ ]:
from glob import glob
from shutil import move

for fname in glob('out_epdn/JPEGImages/*.png'):
  move(fname, "_".join(fname.split("_")[:-1]) + '.png')
  # move(fname, "_")